In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
TRAIN = False

In [2]:
%%capture
!pip install transformers

In [3]:
from transformers import BertTokenizer, BertModel
import torch

In [4]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from tqdm.notebook import tqdm

In [5]:
# this file will run same for training and testing
# training 
if TRAIN:
  hdf5_epath = 'extendable.hdf5'
  !unzip BertNSPInput.zip
  df = pd.read_pickle("BertNSPInput.pk").set_index(['id'])

#testing
else:
  hdf5_epath = 'extendable_test.hdf5'
  !unzip BertNSPInput_test.zip
  df = pd.read_pickle("BertNSPInput_test.pk").set_index(['id'])

Archive:  BertNSPInput_test.zip
  inflating: BertNSPInput_test.pk    


In [6]:
df.head()

,cui,actual_cui,input_ids,token_type_ids,attention_mask
id,,,,,
0,C0019699,C0019699,"[101, 3135, 23435, 1545, 1604, 25892, 1475, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
0,C0019699,C0019699,"[101, 3135, 23435, 1545, 1604, 25892, 1475, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
0,C0019699,C0019699,"[101, 3135, 23435, 1545, 1604, 25892, 1475, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
0,C0019699,C0019699,"[101, 3135, 23435, 1545, 1604, 25892, 1475, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
0,C0019699,C0019699,"[101, 3135, 23435, 1545, 1604, 25892, 1475, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [7]:
if TRAIN:
  x = df.xs(1)
else:
  x = df.xs(0)
for row in x.itertuples():
  print(row.Index)
  print(row._asdict())
  break

0
OrderedDict([('Index', 0), ('cui', 'C0019699'), ('actual_cui', 'C0019699'), ('input_ids', [101, 3135, 23435, 1545, 1604, 25892, 1475, 16899, 10044, 5599, 26752, 1580, 22737, 1571, 5465, 13144, 1571, 1477, 130, 120, 1853, 120, 1949, 1367, 131, 3135, 131, 3135, 6586, 14856, 7111, 2176, 15463, 12917, 1616, 20979, 1174, 141, 6258, 24930, 11234, 14265, 131, 4925, 120, 1853, 120, 1949, 7178, 22130, 131, 20979, 1174, 14856, 7111, 2176, 14265, 131, 1275, 120, 5129, 120, 1949, 145, 6258, 18082, 2069, 3663, 11345, 11629, 9919, 11680, 1942, 15393, 2162, 22680, 12480, 131, 1109, 5351, 1110, 170, 1743, 118, 1214, 118, 1385, 1590, 1150, 1110, 9622, 3112, 1111, 1160, 1201, 119, 1153, 2756, 1114, 1286, 3105, 186, 18413, 6922, 2489, 1112, 1218, 1112, 22882, 1105, 26979, 1158, 1134, 1110, 170, 1263, 118, 2288, 12522, 119, 1153, 1108, 11534, 1107, 1984, 1219, 1103, 3485, 1104, 1123, 2027, 119, 1153, 3711, 1131, 1674, 1136, 1221, 1725, 1131, 1110, 9622, 3112, 119, 1153, 1110, 1121, 4624, 117, 4547, 1125

In [8]:
import gc
torch.cuda.empty_cache()

In [9]:
from torch.nn.functional import softmax
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = BertModel.from_pretrained('dmis-lab/biobert-v1.1',return_dict=True)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

c = model.to(device)


cuda


In [10]:
# with open("cls_embeddings", "w") as f:
#   pass
# with open("is_next", "w") as f:
#   pass

In [11]:
import tables as tb, numpy as np

h5f = tb.open_file(hdf5_epath, mode='a')
dataGroup = h5f.create_group(h5f.root, 'MyData')

myarray = np.zeros((1, 786))
X = h5f.create_earray(dataGroup,"X", obj=myarray)
Y = h5f.create_earray(dataGroup,"Y", obj=np.array([0]))
ID = h5f.create_earray(dataGroup,"id", obj=np.array([0]))
len_cui = 8
CUI = h5f.create_earray(dataGroup,"cui", obj=np.array([' '*len_cui]))
h5f.close()

In [12]:
h5f = tb.open_file(hdf5_epath, mode='r+')

cls_embedding = []
is_next = []
indices = list(set(df.index))
for ind in tqdm(indices):
  rows = df.xs(ind)
  if isinstance(rows, pd.Series):
    print(rows)
    rows = pd.DataFrame(rows).T
    print(rows)
    
  for row in rows.itertuples():
    row = row._asdict()
    encoded = {
      'input_ids': torch.tensor([row['input_ids']], dtype=torch.long),
      'token_type_ids': torch.tensor([row['token_type_ids']], dtype=torch.long),
      'attention_mask': torch.tensor([row['attention_mask']], dtype=torch.long)
    }
    for k in encoded.keys():
      encoded[k] = encoded[k].to(device)
    output = model(**encoded)
    cls_embedding.append(np.resize(output.last_hidden_state[0].cpu().detach().numpy()[0], (1,786)))
    is_next.append(np.array([1 if row['cui'] == row['actual_cui'] else 0]))
    h5f.root.MyData.id.append([row['Index']])
    h5f.root.MyData.cui.append([row['cui']])
  for _ in cls_embedding:
    h5f.root.MyData.X.append(_)
  for _ in is_next:
    h5f.root.MyData.Y.append(_)
  

  del cls_embedding, is_next
  cls_embedding = []
  is_next = []
h5f.close()

cui                                                        C4087013
actual_cui                                                 C0086787
input_ids         [101, 1113, 130, 120, 1744, 1133, 1131, 11519,...
token_type_ids    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 30, dtype: object
         cui  ...                                     attention_mask
30  C4087013  ...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...

[1 rows x 5 columns]
cui                                                        C4087013
actual_cui                                                 C0086787
input_ids         [101, 185, 15342, 20695, 25347, 11876, 119, 18...
token_type_ids    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 66, dtype: object
         cui  ...                                     attention_mask
66  C4087013  ...  [1, 1, 1, 1, 1, 1, 1, 1,

In [16]:
if TRAIN:
  !cp extendable.hdf5 /content/drive/MyDrive/extendable.hdf5
else:
  !cp extendable_test.hdf5 /content/drive/MyDrive/extendable_test.hdf5

In [ ]:
h5f = tb.open_file(hdf5_epath, mode='r')
X = np.array([_ for _ in h5f.root.MyData.X[:]])
Y = np.array([_ for _ in h5f.root.MyData.Y[:]])
ID = np.array([_ for _ in h5f.root.MyData.id[:]])
CUI = np.array([_ for _ in h5f.root.MyData.cui[:]])
h5f.close()

ValueError: ignored